In [1]:
#working on python 3.6.15
# Vertebral column classification
# Keras 2.1.5 TensorFlow 1.7.0 Anaconda3 4.1.1
# raw data looks like:

#81.75441933,20.12346562,70.56044038,61.63095371,119.4250857,55.50688907,0
#51.31177106,8.875541276,56.99999999,42.43622979,126.4722584,-2.144043911,1

# Abstract: Data set containing values for six biomechanical features used to classify orthopaedic patients 
# 2 classes (normal or abnormal).
# 0 = abnormal, 1 = normal

#biomechanical feautures:
# @attribute pelvic_incidence numeric
# @attribute pelvic_tilt numeric
# @attribute lumbar_lordosis_angle numeric
# @attribute sacral_slope numeric
# @attribute pelvic_radius numeric
# @attribute degree_spondylolisthesis numeric


import numpy as np
import keras as K

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'  # suppress CPU msg

class MyLogger(K.callbacks.Callback):
  def __init__(self, n):
    self.n = n   # print loss & acc every n epochs

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.n == 0:
      curr_loss =logs.get('loss')
      curr_acc = logs.get('acc') * 100
      print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % \
        (epoch, curr_loss, curr_acc))

def main():
  print("\nVerify if spine is normal dataset example \n")
  np.random.seed(1)

  # 1. load data
  print("Loading data into memory ")
  train_file = "train.txt"
  test_file = "test.txt"

  train_x = np.loadtxt(train_file, delimiter=',',
    usecols=[0,1,2,3,4,5], dtype=np.float32)
  train_y = np.loadtxt(train_file, delimiter=',',
    usecols=[6], dtype=np.float32)
  test_x = np.loadtxt(test_file, delimiter=',', 
    usecols=[0,1,2,3,4,5], dtype=np.float32)
  test_y =np.loadtxt(test_file, delimiter=',',
    usecols=[6], dtype=np.float32)

  
  norm_train_X=[]

  norm_test_X=[]

  
  for i in range (len(train_x[0])):
    norm_train_X.append((train_x[:,i]-min(train_x[:,i])) / (max(train_x[:,i])-min(train_x[:,i])))
    norm_test_X.append((test_x[:,i]-min(test_x[:,i])) / (max(test_x[:,i])-min(test_x[:,i])))
 
  t_norm_train_X=np.array(norm_train_X).transpose()
  t_norm_test_X=np.array(norm_test_X).transpose()
  print(t_norm_train_X)

  # 2. define 4-(x-x)-1 deep NN model
  print("Creating 4-(8-8)-1 binary NN classifier \n")
  my_init = K.initializers.glorot_uniform(seed=1)
  model = K.models.Sequential()
  model.add(K.layers.Dense(units=8, input_dim=6,
    activation='tanh', kernel_initializer=my_init)) 
  model.add(K.layers.Dense(units=8, activation='tanh',
    kernel_initializer=my_init)) 
  model.add(K.layers.Dense(units=1, activation='sigmoid',
    kernel_initializer=my_init)) 

  # 3. compile model
  simple_sgd = K.optimizers.SGD(lr=0.01)  
  model.compile(loss='binary_crossentropy',
    optimizer=simple_sgd, metrics=['accuracy'])  

  # 4. train model
  max_epochs = 1000
  my_logger = MyLogger(n=50)
  h = model.fit(t_norm_train_X, train_y, batch_size=32,
    epochs=max_epochs, verbose=0, callbacks=[my_logger]) 

  # 5. evaluate model
  np.set_printoptions(precision=6, suppress=True)
  eval_results = model.evaluate(t_norm_test_X,test_y, verbose=0) 
  print("\nLoss, accuracy on test data: ")
  print("%0.4f %0.2f%%" % (eval_results[0], \
eval_results[1]*100))

  # 6. save model
  # mp = ".\\Models\\vcolumn_model.h5"
  # model.save(mp)

  # 7. make a prediction
  inpts = np.array([[0.5, 0.5, 0.5, 0.5,0.5,0.5]], dtype=np.float32)
  pred = model.predict(inpts)
  print("\nPredicting normal vertebral column for: ")
  print(inpts)
  print("Probability that class = 1 (normal):")
  print(pred)

if __name__=="__main__":
  main()


Using TensorFlow backend.
c:\Users\Claudiu\mambaforge\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Claudiu\mambaforge\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Claudiu\mambaforge\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\Claudiu\mambaforge\envs\py36\lib


Banknote authentication dataset example 

Loading data into memory 
[[0.35567322 0.61110795 0.43351611 0.20657583 0.477737   0.04220629]
 [0.12741129 0.24461868 0.26514736 0.17564361 0.5133559  0.02930886]
 [0.28639668 0.6383572  0.3883895  0.12612043 0.5724287  0.03142521]
 ...
 [0.4514367  0.4711969  0.5419044  0.37026435 0.44254383 0.02676807]
 [0.40955412 0.39015022 0.5650486  0.3716718  0.573712   0.11689572]
 [0.16448937 0.47275075 0.24209106 0.09414084 0.5889282  0.01807188]]
Creating 4-(8-8)-1 binary NN classifier 

epoch =    0  loss = 0.583952  acc = 72.43%
epoch =   50  loss = 0.545912  acc = 72.43%
epoch =  100  loss = 0.519515  acc = 71.19%
epoch =  150  loss = 0.499255  acc = 74.07%
epoch =  200  loss = 0.482440  acc = 77.37%
epoch =  250  loss = 0.469480  acc = 78.60%
epoch =  300  loss = 0.459642  acc = 77.78%
epoch =  350  loss = 0.449080  acc = 78.60%
epoch =  400  loss = 0.438831  acc = 79.42%
epoch =  450  loss = 0.429007  acc = 81.89%
epoch =  500  loss = 0.419470